In [1]:
# pip install psycopg2 pandas

In [1]:
import pandas as pd
import psycopg2
import os

# Database connection parameters
DB_NAME = 'course_management'
DB_USER = 'postgres'
DB_PASSWORD = '1234'
DB_HOST = 'localhost'  
DB_PORT = '5432'  

In [2]:


# Connect to the database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

CSV_DIRECTORY = '../creation/'

In [3]:
def seed_designations():
    designation_df = pd.read_csv(CSV_DIRECTORY + 'designations.csv')
    for index, row in designation_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"Designation\" (\"name\") VALUES (%s) ON CONFLICT (\"name\") DO NOTHING",
            (row['name'],)
        )

def seed_courses():
    course_df = pd.read_csv(os.path.join(CSV_DIRECTORY, 'courses.csv'))
    for index, row in course_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"Course\" (\"title\", \"url\", \"shortIntro\", \"difficulty\", \"language\", \"totalTime\", \"totalModules\", \"created_at\", \"updated_at\") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (\"title\") DO NOTHING",
            (row['title'], row['url'], row['shortIntro'], row['difficulty'], row['language'],
             row['totalTime'], row['totalModules'], row['createdAt'], row['updatedAt'])
        )

def seed_skills():
    skill_df = pd.read_csv(os.path.join('../generator/', 'skills.csv'))
    for index, row in skill_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"Skill\" (\"name\") VALUES (%s) ON CONFLICT (\"name\") DO NOTHING",
            (row['skills'],)
        )

def seed_users():
    user_df = pd.read_csv(os.path.join(CSV_DIRECTORY, 'users.csv'))
    
    for index, row in user_df.iterrows():
        try:
            designation_id = int(row['designationId']) if pd.notnull(row['designationId']) else None
            cursor.execute(
                "INSERT INTO public.\"User\" (\"username\", \"passwordHash\", \"email\", \"gender\", \"role\", \"designationId\", \"phoneNumber\", \"createdAt\", \"updatedAt\") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (\"username\") DO NOTHING",
                (row['username'], row['passwordHash'], row['email'], row['gender'], row['role'],
                 designation_id, row['phoneNumber'], row['createdAt'], row['updatedAt'])
            )
        except Exception as e:
            print(f"Error inserting user at index {index}: {e}")

def seed_employee_progress():
    progress_df = pd.read_csv(os.path.join(CSV_DIRECTORY, 'employee_progress.csv'))
    for index, row in progress_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"EmployeeProgress\" (\"userId\", \"courseId\", \"progressStatus\", \"last_updated\", \"modulesCompleted\", \"certificateProof\") VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (\"userId\", \"courseId\") DO NOTHING",
            (row['userId'], row['courseId'], row['progressStatus'], row['lastUpdated'], row['modulesCompleted'], row['certificateProof'])
        )

def seed_user_skills():
    user_skill_df = pd.read_csv(os.path.join(CSV_DIRECTORY, 'user_skills.csv'))
    for index, row in user_skill_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"UserSkill\" (\"userId\", \"skillId\", \"level\") VALUES (%s, %s, %s) ON CONFLICT (\"userId\", \"skillId\") DO NOTHING",
            (row['userId'], row['skillId'], row['level'])
        )

def seed_designation_skills():
    designation_skill_df = pd.read_csv(os.path.join(CSV_DIRECTORY, 'designation_skills.csv'))
    
    designation_skill_df['designationId'] = designation_skill_df['designationId'].astype(int)
    designation_skill_df['skillId'] = designation_skill_df['skillId'].astype(int)

    for index, row in designation_skill_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"DesignationSkill\" (\"designationId\", \"skillId\") VALUES (%s, %s) ON CONFLICT (\"designationId\", \"skillId\") DO NOTHING",
            (int(row['designationId']), int(row['skillId']))
        )

def seed_course_skills():
    course_skill_df = pd.read_csv(os.path.join(CSV_DIRECTORY, 'course_skills.csv'))
    for index, row in course_skill_df.iterrows():
        cursor.execute(
            "INSERT INTO public.\"CourseSkill\" (\"courseId\", \"skillId\", \"level\") VALUES (%s, %s, %s) ON CONFLICT (\"courseId\", \"skillId\") DO NOTHING",
            (row['courseId'], row['skillId'], row['level'])
        )


In [5]:
# Call the seed functions
try:
    seed_designations()
    seed_courses()
    seed_skills()
    
    seed_users()
    seed_employee_progress()
    seed_user_skills()

    seed_designation_skills()
    seed_course_skills()


    # Commit the transaction
    conn.commit()
    print("Seeding completed successfully!")

except Exception as e:
    print(f"Error during seeding: {e}")
    conn.rollback()  # Rollback on error


Seeding completed successfully!


In [ ]:
cursor.close()
conn.close()